# GraalVM and BeakerX

## Why care about accessibility?

Q: Why not ["KISS"](https://en.wikipedia.org/wiki/KISS_principle) and make things work _really well_ on one platform (e.g. linux-amd64+python3)?
<br>A: Because we want to stop [reinventing wheels](https://en.wikipedia.org/wiki/Reinventing_the_wheel).


## Why do we keep rewriting things?

* Accessibility - Code does not run on my target platform(s)
* Licensing - The license is too restrictive
* Performance - The program is not fast enough
* Functionality - The program is not powerful enough
* Quality - The program is brittle, buggy, lacks reproducibility, etc.
* Complexity - The code is too hard to understand and/or change
* (Footprint? i.e. "overkill" or "bloat" but is this really a good reason?)
* (Fun. We like coding.)

# Why is Java awesome?

* Write once, run anywhere
* Great performance thanks to the Just-In-Time (JIT) compiler
* Great standard library
* Huge collection of third party libraries
* Awesome dependency management -- great for reproducible science

# Why does Java suck?
* JIT compilation imposes warmup time
* JIT compilation imposes memory overhead
* Java language is verbose
    * Java code is often harder to read and understand than e.g. Python code
    * There are interpreted scripting languages on the JVM (e.g. Jython, JRuby, Groovy) but they suffer performance-wise.

# What is GraalVM?

"One VM to rule them all"—a performant polyglot virtual machine

https://www.graalvm.org/

# What can GraalVM do?

[Top 10 things GraalVM can do](https://github.com/chrisseaton/graalvm-ten-things) from Chris Seaton (Graal team's Ruby lead)

## 1. High-performance modern Java

* Graal is a Just-In-Time (JIT) compiler written in Java. Easier to develop than C/C++.

* [JVM Compiler Interface](http://openjdk.java.net/jeps/243) (JVMCI) framework enables alternative JIT implementations.

* [More advanced than the traditional C1 (client) and C2 (server) JIT](http://chrisseaton.com/truffleruby/jokerconf17/) implementations.

* Advanced optimizations, like partial [escape analysis](https://en.wikipedia.org/wiki/Escape_analysis).

* Twitter has [switched to GraalVM in production](https://twitter.com/christhalinger/status/986290456718868482) (Scala code) for 20% performance boost.

## 2. Low-footprint, fast-startup Java

`native-image`: an Ahead-Of-Time (AOT) compiler
No need to ship the VM.

## 3. Combine JavaScript, Java, Ruby and R (and Python, and etc.)

[Truffle](https://github.com/oracle/graal/tree/vm-1.0.0-rc1/truffle): a framework for easily and effectively implementing programming languages on the VM.

The "JVM" is not just for Java anymore!

* "That library is not available in my language. I need to rewrite it."
* "That language would be the perfect fit for my problem, but we cannot run it in our environment."
* "That problem is already solved in my language, but the language is too slow."

GraalVM aims "to allow developers to freely choose the right language for the task at hand without making compromises." https://www.graalvm.org/docs/reference-manual/polyglot/

[Graal Polyglot API](https://www.graalvm.org/docs/reference-manual/polyglot/) lets you [embed and run code from guest languages in JVM-based host applications](http://www.graalvm.org/docs/graalvm-as-a-platform/embed/).

Recognizes interpreters written using Truffle and can convert Truffle abstract syntax trees (ASTs) into optimized native code, using [partial evaluation](https://en.wikipedia.org/wiki/Partial_evaluation).

## How can it possibly work?
"Because Truffle provides a standard framework for expressing nodes in an abstract syntax tree, calling into another language doesn’t involve any complex hand-written binding layers. Instead, invoking a function simply joins the ASTs of the two languages together. Those two ASTs are then compiled and optimised by Graal as a single unit, meaning any complexity introduced by crossing the language barrier can be analysed and eliminated."
 ([source](https://blog.plan99.net/graal-truffle-134d8f28fb69))

And no marshaling/unmarshaling of data! Shared representation of objects.

## Which languages
- Java (including JVM-based languages: Clojure, Scala, Groovy, Jython, JRuby, etc.)
    - Caveat: Java itself is not implemented on top of Truffle yet. But [Project Metropolis](TODO) is underway to change that.
- JavaScript
- Ruby
- Python (alpha)
- C, C++, Fortran - via LLVM bitcode

__Complexity:__ "To give a feel for how easy it is to write these engines, TruffleJS is only about 80,000 lines of code compared to about 1.7 million for V8." ([source](https://blog.plan99.net/graal-truffle-134d8f28fb69))

__Performance:__ Is competitive. E.g. TruffleRuby is ~8x faster than CRuby 2.x ([source](https://pragtob.wordpress.com/2017/01/24/benchmarking-a-go-ai-in-ruby-cruby-vs-rubinius-vs-jruby-vs-truffle-a-year-later/))

## Which languages are implemented?

In [1]:
import org.graalvm.polyglot.Context
context = Context.newBuilder().allowAllAccess(true).build()
context.getEngine().getLanguages()

In [2]:
println('Hello Groovy')
context.eval("js", "print('Hello JavaScript')")
context.eval("python", "print('Hello Python')")
context.eval("R", "print('Hello R');")
context.eval("ruby", "puts('Hello Ruby')")

Hello Groovy
Hello JavaScript
Hello Python
[1] "Hello R"
Hello Ruby


nil

## 4. Run native languages on the JVM

-- node.js native vs jvm mode? Worth clarifying, or no?

## 5. Tools that work across all languages

"The Truffle framework is a kind of nexus for languages and tools."
 -- e.g., Chrome debugger for Python

Debugging
- "Languages such as R and Ruby can be debugged as easily as JavaScript, including stepping through language boundaries during guest language interoperability."
- Chrome DevTools, JVisualVM
- http://www.graalvm.org/docs/reference-manual/tools/
- https://medium.com/graalvm/debugging-polyglot-node-js-ruby-r-apps-with-graalvm-81b1bb2614db

"If you run a standard JVM language, like JRuby, with VisualVM you'll be disappointed in that you'll see the underlying Java objects, rather than any information about your language's objects. If we use the GraalVM version of Ruby instead, VisualVM will recognise the Ruby objects themselves. We need to use the --jvm command to use VisualVM, as it doesn't support the native version of Ruby."

## 6. Extend a JVM-based application

In [3]:
context.eval("python", "[2**n for n in range(0, 16)]")

[1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096, 8192, 16384, 32768]

## 7. Extend a native application
- link to libpolyglot and invoke any/all Truffle languages from C et al

## 8. Java code as a native library

E.g. compile Bio-Formats to native code?? Did not try yet.

## 9. Polyglot in the database

TODO

## 10. Create your own language

Truffle languages are written in Java.

"An advanced visualizer tool lets you explore the compiler's intermediate representation as it passes through optimization stages." ([source](https://blog.plan99.net/graal-truffle-134d8f28fb69))

# Questions

## What are the downsides?
Space/time tradeoffs: warmup time, memory usage, vs. static comp
* JIT = more memory, longer startup and warmup times
* AOT = less memory, fast startup, but less dynamic/extensible and often worse peak performance

## Will it catch on?
Graal's improved JIT already ships with Java 9 and 10.
```
java -XX:+UnlockExperimentalVMOptions \
     -XX:+EnableJVMCI \
     -XX:+UseJVMCICompiler \
     -Djvmci.Compiler=graal ...
```
~30 core developers at Oracle Labs and academic partners.

## But... it's Oracle! What about open source?

Core technologies are copyleft:
* Truffle and Graal SDK = GPLv2 with Classpath exception
* Tools project = GPLv2 with Classpath exception
* TRegex project = GPLv2 with Classpath exception
* The Graal compiler = GPLv2
* Substrate VM = GPLv2

Languages are permissive:
* TruffleRuby = EPLv1 or GPLv2 or LGPLv2.1
* graaljs = UPLv1
* graalpython = UPLv1
* sulong (LLVM) = BSD-3
* FastR = APLv2

Impressed by not only open _licensing_, but also [openness of process](https://imagej.net/Philosophy#Open_source):
* Academic papers and partnerships
* All on GitHub ([oracle/graal](https://github.com/oracle/graal), [graalvm](https://github.com/graalvm/))
* Active [chat room on Gitter](https://gitter.im/graalvm/graal-core)

## What about Python?

org.graalvm.python implementation still alpha quality. (TODO: demo of it sucking)

But it is a development priority of the core team (~30-40 devs):

> "Graal.python is pretty new and its really early days. So even basic language things might break right now. But we are investing a lot of resources in Python support." ([source](https://news.ycombinator.com/item?id=16867048), from 2018-04-18)

In the meantime, we can still invoke Java from Python with shared memory using e.g. pyjnius.
Combined with Graal, this is very powerful: Python + &lt;any-Truffle-language&gt; for free.

### PyPy? Numba? Emscripten?

"Because scripting languages are so slow it's very common to rewrite performance hotspots in dynamically typed programs by hand in C, using the original interpreter's internal API to interact with the scripted code. Perversely, this technique actually makes it harder to speed up the language in general because running real programs often means running their C extensions too, and that's very difficult when those extensions make so many assumptions about the runtime's internals." ([source](https://blog.plan99.net/graal-truffle-134d8f28fb69))

TruffleRuby supports Ruby's C extensions performantly by interpreting the C code (!!) via the LLVM support. ([source](http://chrisseaton.com/truffleruby/cext/))

## What about the browser?
JavaScript in the browser is tied to specific JS implementations (Chrome = V8, Firefox = SpiderMonkey).

It is unclear (to me) if GraalVM's JS implementation will ever be usable in browser... but seems unlikely.

To maximize space and time performance, we still need interprocess shared memory between the browser's JS engine and the backend. But at least now the backend is easier to unify.

## What about Electron?
Possible on the node.js side, but no one has done it yet:

> "It is essential to find version of node and electron that work together. I have found out that electron 2.0.0betaX is using node 8.9.3 and GraalVM 1.0.0rc1 is implementing node 8.9.4 - e.g. these versions shall be replaceable. I haven't found time to change the buildscripts yet, however. Passing `--jvm --polyglot` to the npm invocation that starts electron is not enough ... It is necessary to recompile electron executable with different node sources." [—Jaroslav Tulach](https://gitter.im/graalvm/graal-core?at=5af9897d2df44c2d062787b2), Graal team

## What about mobile?
TODO

## What about shared memory?

Let's try! We'll check the memory usage after each language uses the data.

In [4]:
mbUsed = { System.gc(); return (Runtime.getRuntime().totalMemory() - Runtime.getRuntime().freeMemory()) / 1024 / 1024 }
before = mbUsed()

170.85143280029296875

In [5]:
bigArray = new double[99 * 1000 * 1000]
for (int i=0; i<bigArray.length; i++) bigArray[i] = Math.random()
mbUsed() - before

740.52420043945312500

In [6]:
// We can push a variable directly into a language's global scope.
context.getBindings("js").putMember("bigArray", bigArray)
context.eval("js",
"""
print('JavaScript: array type = ' + typeof(bigArray) + ', length = ' + bigArray.length)
""")
mbUsed() - before

JavaScript: array type = object, length = 99000000


745.61810302734375000

In [7]:
// We can use the polyglot bindings to avoid polluting the language's global namespace.
context.getPolyglotBindings().putMember("bigArray", bigArray)
context.eval("python",
"""
import polyglot
big_array = polyglot.import_value('bigArray')
print('Python: array type = {}, length = {}'.format(str(type(big_array)), len(big_array)))
"""
)
mbUsed() - before

Python: array type = <class 'foreign'>, length = 99000000


756.05783081054687500

## Are the objects _really_ the same?

Let's find out! We'll try doing some array-ish things in various languages.

In [8]:
sum = 0
for (int i=0; i<bigArray.length; i++) sum += bigArray[i]
sum // Bahaha wrongness! Fixed precision.

4.9502350241765894E7

In [9]:
context.eval("js",
"""
sum = Array.from(bigArray).reduce(function(total, x) { return total + x; });
print('JavaScript sum = ' + sum)
""")
mbUsed() - before

TypeError:  only JavaScript objects are supported by this operation

In [10]:
context.eval("python",
"""
sum = sum(big_array)
print('Python sum = {}'.format(sum))
"""
)
mbUsed() - before

Python sum = 4.9502350241765894e+7


762.91050720214843750